In [16]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import cv2
import numpy as np

In [18]:
train_folder = './shopee-product-detection-dataset/train/train/'

In [12]:
def construct_model(num_class):
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))
    x=base_model.output
    # Add some new Fully connected layers to 
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x)
    x = Dropout(0.25)(x)
    x=Dense(512,activation='relu')(x) 
    x = Dropout(0.25)(x)
    preds=Dense(num_class, activation='softmax')(x) #final layer with softmax activation

    model=Model(inputs=base_model.input,outputs=preds)
    return model

In [13]:
train_model = construct_model(42)

In [14]:
train_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [22]:
train_gen = ImageDataGenerator()

In [23]:
train_gen = train_gen.flow_from_directory(train_folder, (224, 224))

Found 105392 images belonging to 42 classes.


In [25]:
train_model.compile(optimizer=Adam(1e-4), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = train_model.fit_generator(train_gen, 1700, 50)

Epoch 1/50
1700/1700 [==============================] - 1138s 669ms/step - loss: 1.5867 - accuracy: 0.5735
Epoch 2/50
1700/1700 [==============================] - 877s 516ms/step - loss: 1.1879 - accuracy: 0.6774
Epoch 3/50
1700/1700 [==============================] - 403s 237ms/step - loss: 0.9754 - accuracy: 0.7297
Epoch 4/50
1700/1700 [==============================] - 406s 239ms/step - loss: 0.9383 - accuracy: 0.7434
Epoch 5/50
1700/1700 [==============================] - 404s 237ms/step - loss: 0.8003 - accuracy: 0.7763
Epoch 6/50
1700/1700 [==============================] - 547s 322ms/step - loss: 0.7774 - accuracy: 0.7827
Epoch 7/50
1700/1700 [==============================] - 403s 237ms/step - loss: 0.6672 - accuracy: 0.8099
Epoch 8/50
1700/1700 [==============================] - 499s 294ms/step - loss: 0.6555 - accuracy: 0.8121
Epoch 9/50
1700/1700 [==============================] - 403s 237ms/step - loss: 0.5685 - accuracy: 0.8332
Epoch 10/50
1700/1700 [======================

In [27]:
train_model.sample_weights('only_weights.h5')

TypeError: 'list' object is not callable

In [28]:
model_json = train_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
train_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [29]:
train_model.save('model_and_weights.h5')

In [32]:
from keras.models import model_from_json
from keras.models import load_model


In [ ]:
json_file = open('model_num.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")